In [150]:
import pandas as pd
import json

In [151]:
# READING THE LOG FILES

path1 = 'exec_logs.log'
path2 = 'market_logs.log'

exec_logs = open(path1,'r')
market_logs = open(path2,'r')

result = {}
i = 0
for line in exec_logs:
    columns = line.split(',')
    data=[]
    for c in columns:
        c = c.split(' ')
        for x in c:
            data.append(x)
    result[i] = data
    i += 1
j = json.dumps(result)

exec_logs_df = pd.read_json(j, orient='index')

result = {}
i = 0
for line in market_logs:
    columns = line.split(',')
    data=[]
    for c in columns:
        c = c.split(' ')
        for x in c:
            data.append(x)
    result[i] = data
    i += 1
j=json.dumps(result)

market_logs_df = pd.read_json(j, orient='index')

In [182]:
exec_logs_df.tail(5)

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
8677,eLog:,order,exec,at,1627626326117581568,with,id,146366994331992064,,side,...,1,,volume,left,on,level,564,,delta_execsend,2381312
8678,eLog:,order,exec,at,1627626397873668608,with,id,148337319168966656,,side,...,1,,volume,left,on,level,331,,delta_execsend,1296281088
8679,eLog:,order,exec,at,1627626412584847104,with,id,146929944285413376,,side,...,1,,volume,left,on,level,97,,delta_execsend,14703859968
8680,eLog:,order,exec,at,1627626473991536128,with,id,149181744099098624,,side,...,1,,volume,left,on,level,10,,delta_execsend,12134184704
8681,eLog:,order,exec,at,1627626512859773440,with,id,148900269122387968,,side,...,1,,volume,left,on,level,15,,delta_execsend,14734981632


In [153]:
market_logs_df.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,mLog:,market,trd,at,1622505601191796224,with,side,-1,,price,1108.9,,traded,volume,36,then,book,became,6@1108.7x1108.8@104\n
1,mLog:,market,trd,at,1622505601222248192,with,side,-1,,price,1108.7,,traded,volume,17,then,book,became,10@1108.5x1108.6@23\n
2,mLog:,market,trd,at,1622505601364445696,with,side,-1,,price,1108.5,,traded,volume,20,then,book,became,16@1108.3x1108.6@23\n
3,mLog:,market,trd,at,1622505601405326080,with,side,1,,price,1108.6,,traded,volume,1,then,book,became,16@1108.3x1108.6@22\n
4,mLog:,market,trd,at,1622505601406994688,with,side,-1,,price,1108.3,,traded,volume,16,then,book,became,18@1108.2x1108.3@12\n


In [193]:
# CLEANING THE TABLES AND RENAMING COLUMNS

exec_logs = exec_logs_df.iloc[:, [4, 7, 10, 13, 17, 23, 26]]
exec_logs.columns = ['Order exec at', 'id', 'exec side', 'exec price', 'exec traded volume', 'volume left on level', 'delta_execsend']
market_logs = market_logs_df.iloc[:, [4, 7, 10, 14, 18]]
market_logs.columns = ['Market trade at', 'side', 'price', 'traded volume', 'then book became']

In [194]:
exec_logs.head(5)

,Order exec at,id,exec side,exec price,exec traded volume,volume left on level,delta_execsend
0,1622505601222248192,281481419161600,1,1108.6,1,10,30451968
1,1622505601424113408,562956395872256,-1,1108.4,1,22,17118720
2,1622505601604667904,844431372582912,1,1108.3,1,2,153866496
3,1622505602982201088,1125906349293568,-1,1108.4,1,1,153371136
4,1622505603600579584,1407381326004224,1,1108.3,1,6,618378496


In [195]:
market_logs.head(5)

,Market trade at,side,price,traded volume,then book became
0,1622505601191796224,-1,1108.9,36,6@1108.7x1108.8@104\n
1,1622505601222248192,-1,1108.7,17,10@1108.5x1108.6@23\n
2,1622505601364445696,-1,1108.5,20,16@1108.3x1108.6@23\n
3,1622505601405326080,1,1108.6,1,16@1108.3x1108.6@22\n
4,1622505601406994688,-1,1108.3,16,18@1108.2x1108.3@12\n


In [196]:
# TURNING EPOCH NANOSECONDS TIME TO SECONDS, ROUNDED TO NEAREST SECOND

exec_logs['Order exec at, sec'] = (exec_logs['Order exec at']/1000000000).round()
exec_logs.drop(columns=['Order exec at'], inplace=True)
market_logs['Market trade at, sec'] = (market_logs['Market trade at']/1000000000).round()
market_logs.drop(columns=['Market trade at'], inplace=True)

/var/folders/7v/sydf4rx94t73k3tg85fd1fjw0000gn/T/ipykernel_5686/4188839271.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec_logs['Order exec at, sec'] = (exec_logs['Order exec at']/1000000000).round()
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/var/folders/7v/sydf4rx94t73k3tg85fd1fjw0000gn/T/ipykernel_5686/4188839271.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [216]:
# EXCTRACTING AND CALCULATING MARKET MID PRICE

def func(string):
    market_ask = string.split('@')[1].split('x')[0]
    market_bid = string.split('@')[1].split('x')[1]
    mid_price = (float(market_ask) + float(market_bid))/2
    return mid_price

market_logs['mid price'] = market_logs['then book became'].apply(func)
market_logs.head(5)

/var/folders/7v/sydf4rx94t73k3tg85fd1fjw0000gn/T/ipykernel_5686/3200360467.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  market_logs['mid price'] = market_logs['then book became'].apply(func)


,side,price,traded volume,then book became,"Market trade at, sec",mid price,"Market trade at, sec -5"
0,-1,1108.9,36,6@1108.7x1108.8@104\n,1.622506e+09,1108.75,1.622506e+09
1,-1,1108.7,17,10@1108.5x1108.6@23\n,1.622506e+09,1108.55,1.622506e+09
2,-1,1108.5,20,16@1108.3x1108.6@23\n,1.622506e+09,1108.45,1.622506e+09
3,1,1108.6,1,16@1108.3x1108.6@22\n,1.622506e+09,1108.45,1.622506e+09
4,-1,1108.3,16,18@1108.2x1108.3@12\n,1.622506e+09,1108.25,1.622506e+09


In [199]:
# SUBTRACTING 5 SECONDS TO MERGE MARKET AND EXEC TABLES ON +5 SECONDS TIME DIFFERENCE

market_logs['Market trade at, sec -5'] = market_logs['Market trade at, sec'] - 5

/var/folders/7v/sydf4rx94t73k3tg85fd1fjw0000gn/T/ipykernel_5686/1071114781.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  market_logs['Market trade at, sec -5'] = market_logs['Market trade at, sec'] - 5


In [217]:
merged_df = exec_logs.merge(market_logs, how='left', left_on='Order exec at, sec', right_on='Market trade at, sec -5')
merged_df.head(5)

,id,exec side,exec price,exec traded volume,volume left on level,delta_execsend,"Order exec at, sec",side,price,traded volume,then book became,"Market trade at, sec",mid price,"Market trade at, sec -5"
0,281481419161600,1,1108.6,1,10,30451968,1.622506e+09,-1.0,1108.7,1.0,4@1108.6x1108.8@5\n,1.622506e+09,1108.70,1.622506e+09
1,281481419161600,1,1108.6,1,10,30451968,1.622506e+09,-1.0,1108.7,5.0,4@1108.6x1108.7@25\n,1.622506e+09,1108.65,1.622506e+09
2,281481419161600,1,1108.6,1,10,30451968,1.622506e+09,1.0,1108.7,1.0,4@1108.6x1108.7@64\n,1.622506e+09,1108.65,1.622506e+09
3,562956395872256,-1,1108.4,1,22,17118720,1.622506e+09,-1.0,1108.7,1.0,4@1108.6x1108.8@5\n,1.622506e+09,1108.70,1.622506e+09
4,562956395872256,-1,1108.4,1,22,17118720,1.622506e+09,-1.0,1108.7,5.0,4@1108.6x1108.7@25\n,1.622506e+09,1108.65,1.622506e+09


In [218]:
# CALCULATING dmid5

merged_df['dmid5'] = (merged_df['mid price'] - merged_df['price'])*merged_df['exec side']

In [219]:
# CALCULATING CORRELATION (ANSWER)

corr_df = merged_df[['dmid5', 'delta_execsend']].dropna()
corr_df.reset_index(inplace=True, drop=True)
corr_df.head(5)

,dmid5,delta_execsend
0,-2.273737e-13,30451968
1,-5.000000e-02,30451968
2,-5.000000e-02,30451968
3,2.273737e-13,17118720
4,5.000000e-02,17118720


In [220]:
# CALCULATING CORRELATION (ANSWER)

corr_answer = corr_df.corr()
print("Correlation =",corr_answer.iloc[1, 0])

Correlation = 0.006568219319472857
